## Traveling Salesperson (TSP) pyomo formulation

Example modifed from:https://medium.com/analytics-vidhya/model-and-solution-of-the-traveling-salesman-problem-with-python-and-pyomo-db45f2631e8c


Assume there are n cities with distance/cost from city i to j of $c_{ij}$. $x_{ij}$ is the path from city i to city j. And we define dummy variable to be $u_i$. The problem can be written as follows:

![TSP Formulas](TSPFormulation.png)

The first set of equalities requires that each city is arrived at from exactly one other city, and the second set of equalities requires that from each city there is a departure to exactly one other city. The last constraints enforce that there is only a single tour covering all cities, and not two or more disjointed tours that only collectively cover all cities

In [ ]:
import pandas as pd
import pyomo.environ as pe

Now we will create the pyomo model by first reading in the cost matrix.

In [ ]:
cost_matrix = pd.read_excel('tsp-data.xlsx', header = None)
N_cities = len(cost_matrix)
cost_matrix

Next we will define the indexes for the cities and the dummy variable U.

In [ ]:
model = pe.ConcreteModel()

#Indexes for the cities
model.M = pe.RangeSet(N_cities)                
model.N = pe.RangeSet(N_cities)

#Index for the dummy variable U
model.U = pe.RangeSet(2, N_cities)

Next we will create our decision variables which will hold our grid of cities to visit and our dummy variable.

In [ ]:
#Decision variables xij
#Note this creates a grid of N x M decision variables
model.x = pe.Var(model.N, model.M, domain = pe.Binary)

#Dummy variable ui
model.u = pe.Var(model.N, domain = pe.NonNegativeIntegers, bounds =(0, N_cities - 1))

We next define a parameter to hold the cost matrix.

In [ ]:
#Cost Matrix cij
model.c = pe.Param(model.N, model.M, initialize = lambda model, i, j: cost_matrix[i - 1][j - 1])

Then we define the objective function. Notice how we can loop through the n x m cost values.

In [ ]:
def obj_func(model):
    return sum(model.x[i, j] * model.c[i, j] for i in model.N for j in model.M)
model.obj = pe.Objective(rule = obj_func, sense = pe.minimize)

The first constraint ensures that only 1 leaves each city can be formulated in the following way:

In [ ]:
def rule_const1(model, M):
    return sum(model.x[i, M] for i in model.N if i != M ) == 1
model.const1 = pe.Constraint(model.M, rule = rule_const1)

The second constraint ensures that each city receives only 1: 

In [ ]:
def rule_const2(model, N):
    return sum(model.x[N, j] for j in model.M if j != N) == 1
model.rest2 = pe.Constraint(model.N, rule = rule_const2)

The third and last constraint is the one that enforces that there is only a single tour covering all cities, and not two or more disjointed tours that only collectively cover all cities.


In [ ]:
def rule_const3(model, i, j):
    if i!=j: 
        return model.u[i] - model.u[j] + model.x[i,j] * N_cities <= N_cities-1
    else:
        #A rule type function must provide a Pyomo Object, so that’s why I had to write this else
        return model.u[i] - model.u[i] == 0 
    
model.rest3 = pe.Constraint(model.U, model.N, rule = rule_const3)

In [ ]:
#Prints the entire model
model.pprint()

Now we are ready to solve the model.

In [ ]:
opt = pe.SolverFactory('glpk')
result = opt.solve(model)
print(result.solver.status, result.solver.termination_condition)

In [ ]:
# optimal objective value
print()
obj_val = model.obj.expr()
print(f'optimal objective value = {obj_val:.2f}')

dv_keys = list(model.x.keys())
for key in dv_keys:
    if model.x[key]() != 0:
        print(key,'--', model.x[key]())

In [ ]:
print(f'Therefore, the optimal tour is: 1  5  6  4  3  2  1 with a distance of {obj_val}')